<a href="https://colab.research.google.com/github/Abhi-08-28/Document-Summarizer/blob/main/Document_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  !nvidia-smi

Sat Dec  2 08:17:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.2 MB/s

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.3 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 14.3 MB/s eta 0:00:00
  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [7]:
!wget https://zenodo.org/records/7152317/files/dataset.zip?download=1
!unzip dataset.zip?download=1

Streaming output truncated to the last 5000 lines.
  inflating: dataset/IN-Abs/train-data/summary/4620.txt  
  inflating: dataset/IN-Abs/train-data/summary/463.txt  
  inflating: dataset/IN-Abs/train-data/summary/4630.txt  
  inflating: dataset/IN-Abs/train-data/summary/4631.txt  
  inflating: dataset/IN-Abs/train-data/summary/4643.txt  
  inflating: dataset/IN-Abs/train-data/summary/4647.txt  
  inflating: dataset/IN-Abs/train-data/summary/4652.txt  
  inflating: dataset/IN-Abs/train-data/summary/4667.txt  
  inflating: dataset/IN-Abs/train-data/summary/4668.txt  
  inflating: dataset/IN-Abs/train-data/summary/4682.txt  
  inflating: dataset/IN-Abs/train-data/summary/4690.txt  
  inflating: dataset/IN-Abs/train-data/summary/4693.txt  
  inflating: dataset/IN-Abs/train-data/summary/4697.txt  
  inflating: dataset/IN-Abs/train-data/summary/4711.txt  
  inflating: dataset/IN-Abs/train-data/summary/4712.txt  
  inflating: dataset/IN-Abs/train-data/summary/4715.txt  
  inflating: dataset/I

In [8]:
import pandas as pd

In [9]:
dataframe=pd.read_csv(r'/content/dataset/IN-Abs/train-data/stats-IN-train.txt', sep='\t',header=None)

In [10]:
dataframe

,0,1,2,3,4
0,6712.txt,1385,509,0.37,42
1,2858.txt,4867,708,0.15,134
2,6466.txt,6074,1134,0.19,187
3,6818.txt,2683,850,0.32,100
4,5824.txt,4182,1173,0.28,133
...,...,...,...,...,...
7025,4354.txt,263,150,0.57,11
7026,2831.txt,1047,301,0.29,44
7027,190.txt,3662,416,0.11,99
7028,3771.txt,3499,922,0.26,104


In [11]:
from datasets import load_dataset

dataset = load_dataset('/content/dataset/IN-Abs/train-data/summary')


split_lengths = [len(dataset[split]) for split in dataset]


print(f"Split lengths: {split_lengths}")


print(f"Features: {dataset['train'].column_names}")

print("\nDialogue:")
print(dataset['train']['text'])


Resolving data files:   0%|          | 0/7030 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
from datasets import load_dataset


dataset_name = 'dataset'
dataset = load_dataset('/content/dataset/IN-Abs/train-data/judgement')


split_lengths = [len(dataset[split]) for split in dataset]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nFirst Example from the training set:")
print(dataset['train'][0])


Resolving data files:   0%|          | 0/7030 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Split lengths: [954967]
Features: ['text']

First Example from the training set:
{'text': 'Appeal No. LXVI of 1949.'}


In [18]:
from datasets import load_dataset

# If 'stats-IN-train.txt' is a file containing your dataset, you should use the 'text' loader
# and specify the path to your file like this:

file_path = '/content/dataset/IN-Abs/train-data/stats-IN-train.txt'  # Update the path to where your file is located
dataset = load_dataset('text', data_files= '/content/dataset/IN-Abs/train-data/stats-IN-train.txt')

# If 'stats-IN-train.txt' is a dataset name from an online repository (which seems unlikely with a `.txt` extension),
# you would load it like this:

# dataset_name = 'stats-IN-train.txt'
# dataset = load_dataset(dataset_name)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
!wget "Desktop\project\summarizer\Document-Summarizer\dataset.zip_download=1"
!unzip dataset.zip

--2023-12-02 08:28:07--  http://desktop%5Cproject%5Csummarizer%5Cdocument-summarizer%5Cdataset.zip_download=1/
Resolving desktop\\project\\summarizer\\document-summarizer\\dataset.zip_download=1 (desktop\\project\\summarizer\\document-summarizer\\dataset.zip_download=1)... failed: Name or service not known.
wget: unable to resolve host address ‘desktop\\project\\summarizer\\document-summarizer\\dataset.zip_download=1’
unzip:  cannot find or open dataset.zip, dataset.zip.zip or dataset.zip.ZIP.


In [20]:
import pandas as pd

In [21]:
file_path = '/content/drive/Othercomputers/My Laptop/dataset/dataset/IN-Abs/train-data/judgement/1002.txt'  # Replace with the actual file path
with open(file_path, 'r') as file:
    text_contents = file.read()

print(text_contents)

Appeals Nos. 155 to 160 of 1956.
Appeals from the judgments and orders of the Bombay High Court dated July 6, 1954, in Special Civil Applications Nos. 393, 395, 409 and 632 of 1954; July 19, 1954, in Special Civil Application No. 1205 of 1954; and July 30, 1954, in Special Civil Application No. 1309 of 1954.
Purshottam Trikamdas, V. M. Limaye, E. Udayaratnam and section section Shukla, for the appellants.
H. N. Sanyal, Additional Solicitor General of India, N. P. Nathwani, K. L. Hathi and R. H. Dhebar, for the respondents. 1960.
October 3.
The Judgment of the Court was delivered by WANCHOO J.
These six appeals on a certificate granted by the Bombay High Court raise a common question as to the constitutionality of the Bombay Personal Inams Abolition Act, No. XLII of 1953, (hereinafter called the Act) and will be disposed of by this judgment.
The appellants hold personal inams which are covered by Bombay Acts Nos. 11 and VII of 1863.
The Act was attacked on a number of grounds in the Hig

In [22]:
file_path = '/content/drive/Othercomputers/My Laptop/dataset/dataset/IN-Abs/train-data/judgement/1002.txt'  # Replace with the actual file path
num_lines = 5  # Number of lines to display

with open(file_path, 'r') as file:
    text_lines = [next(file) for _ in range(num_lines)]

for line in text_lines:
    print(line)


Appeals Nos. 155 to 160 of 1956.

Appeals from the judgments and orders of the Bombay High Court dated July 6, 1954, in Special Civil Applications Nos. 393, 395, 409 and 632 of 1954; July 19, 1954, in Special Civil Application No. 1205 of 1954; and July 30, 1954, in Special Civil Application No. 1309 of 1954.

Purshottam Trikamdas, V. M. Limaye, E. Udayaratnam and section section Shukla, for the appellants.

H. N. Sanyal, Additional Solicitor General of India, N. P. Nathwani, K. L. Hathi and R. H. Dhebar, for the respondents. 1960.

October 3.



In [23]:
file_path = '/content/drive/Othercomputers/My Laptop/dataset/dataset/IN-Abs/train-data/judgement/1002.txt'
with open(file_path, 'r') as file:
    text_contents = file.read()

print(text_contents)



Appeals Nos. 155 to 160 of 1956.
Appeals from the judgments and orders of the Bombay High Court dated July 6, 1954, in Special Civil Applications Nos. 393, 395, 409 and 632 of 1954; July 19, 1954, in Special Civil Application No. 1205 of 1954; and July 30, 1954, in Special Civil Application No. 1309 of 1954.
Purshottam Trikamdas, V. M. Limaye, E. Udayaratnam and section section Shukla, for the appellants.
H. N. Sanyal, Additional Solicitor General of India, N. P. Nathwani, K. L. Hathi and R. H. Dhebar, for the respondents. 1960.
October 3.
The Judgment of the Court was delivered by WANCHOO J.
These six appeals on a certificate granted by the Bombay High Court raise a common question as to the constitutionality of the Bombay Personal Inams Abolition Act, No. XLII of 1953, (hereinafter called the Act) and will be disposed of by this judgment.
The appellants hold personal inams which are covered by Bombay Acts Nos. 11 and VII of 1863.
The Act was attacked on a number of grounds in the Hig

In [24]:
file_path = '/content/drive/Othercomputers/My Laptop/dataset/dataset/IN-Abs/train-data/judgement/1002.txt'
with open(file_path, 'r') as file:
    text_content = file.read()

print(text_content)



Appeals Nos. 155 to 160 of 1956.
Appeals from the judgments and orders of the Bombay High Court dated July 6, 1954, in Special Civil Applications Nos. 393, 395, 409 and 632 of 1954; July 19, 1954, in Special Civil Application No. 1205 of 1954; and July 30, 1954, in Special Civil Application No. 1309 of 1954.
Purshottam Trikamdas, V. M. Limaye, E. Udayaratnam and section section Shukla, for the appellants.
H. N. Sanyal, Additional Solicitor General of India, N. P. Nathwani, K. L. Hathi and R. H. Dhebar, for the respondents. 1960.
October 3.
The Judgment of the Court was delivered by WANCHOO J.
These six appeals on a certificate granted by the Bombay High Court raise a common question as to the constitutionality of the Bombay Personal Inams Abolition Act, No. XLII of 1953, (hereinafter called the Act) and will be disposed of by this judgment.
The appellants hold personal inams which are covered by Bombay Acts Nos. 11 and VII of 1863.
The Act was attacked on a number of grounds in the Hig

In [29]:
!pip install --upgrade transformers


In [33]:
pip install load_from_disk

ERROR: Could not find a version that satisfies the requirement load_from_disk (from versions: none)
ERROR: No matching distribution found for load_from_disk


In [38]:
from transformers import pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.save_pretrained("/path/to/save/tokenizer")

loaded_tokenizer = AutoTokenizer.from_pretrained("/path/to/save/tokenizer")


In [42]:
from transformers import load_from_disk

ImportError: ignored

In [36]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [39]:
from datasets import load_dataset
file_path = "/content/drive/Othercomputers/My Laptop/dataset/dataset/IN-Abs/train-data/judgement/1002.txt"

try:
    dataset = load_dataset('text', data_files={'train': /content/drive/Othercomputers/My Laptop/dataset/dataset/IN-Abs/train-data/judgement/1002.txt})
except FileNotFoundError as e:
    print(f"File not found: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


SyntaxError: ignored

In [ ]:
dataset_summarizer["train"]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_summarizer["train"])

In [ ]:
trainer.train()


In [ ]:
def safe_indexing(data, index):

    if isinstance(data, list) and 0 <= index < len(data):
        return data[index]
    else:

        print(f"Index {index} is out of bounds for the data structure with size {len(data)}.")
        return None

data = [1, 2, 3]
index_to_access = 47


element = safe_indexing(data, index_to_access)
if element is not None:

    pass


In [40]:
trainer.train()

NameError: ignored

In [41]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

NameError: ignored